In [171]:
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd

In [160]:
# 神奈川全域 新着順
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=14&sc=14101&sc=14102&sc=14103&sc=14104&sc=14105&sc=14106&sc=14107&sc=14108&sc=14109&sc=14110&sc=14111&sc=14112&sc=14113&sc=14114&sc=14115&sc=14116&sc=14117&sc=14118&sc=14131&sc=14132&sc=14133&sc=14134&sc=14135&sc=14136&sc=14137&sc=14151&sc=14152&sc=14153&sc=14201&sc=14203&sc=14204&sc=14205&sc=14206&sc=14207&sc=14208&sc=14210&sc=14211&sc=14212&sc=14213&sc=14214&sc=14215&sc=14216&sc=14217&sc=14218&sc=14300&sc=14320&sc=14340&sc=14360&sc=14400&sc=14380&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=09&page=2"

# &page=2

In [161]:
r = requests.get(url)

In [162]:
r

<Response [200]>

In [163]:
soup = BeautifulSoup(r.text, 'lxml')

In [168]:
df_ = pd.DataFrame([url, soup], index=['url','soup'])

In [169]:
df_

,0
url,https://suumo.jp/jj/chintai/ichiran/FR301FC001...
soup,"[html, [if lt IE 7]><html lang=""ja"" class=""ie""..."


In [39]:
properties = soup.find_all('div', class_='cassetteitem')

In [40]:
len(properties)

30

In [41]:
len(
    properties[0].find_all(class_='cassetteitem_other')[0].find_all('tbody')[0].find_all('td')
)

9

In [42]:
properties[0].find_all(class_='cassetteitem_other')[0].find_all('tbody')[0].find_all('td')[2]

<td>
											4階</td>

### 項目の確認作業

In [66]:
# 一件目
each_property = properties[0]
# 複数案内がある場合はそれぞれに対して項目を抽出する
proposes = each_property.find_all(class_='cassetteitem_other')[0]
# ある一つの案内について
each_propose = proposes.find_all('tbody')[0]
# 物件情報
infos = each_propose.find_all('td')

In [67]:
infos[2]

<td>
											4階</td>

In [170]:
floor = infos[2].text.split('\r\n\t\t\t\t\t\t\t\t\t\t\t')[1]
rent_fee = infos[3].find_all(class_='cassetteitem_price--rent')[0].text
manegement_fee = infos[3].find_all(class_='cassetteitem_price--administration')[0].text
deposit = infos[4].find_all(class_='cassetteitem_price--deposit')[0].text
gratuity = infos[4].find_all(class_='cassetteitem_price--gratuity')[0].text
layout = infos[5].find_all(class_='cassetteitem_madori')[0].text
exclusive_area = infos[5].find_all(class_='cassetteitem_menseki')[0].text
detail_link = infos[8].a['href']

<td class="ui-text--midium ui-text--bold">
<a class="js-cassette_link_href cassetteitem_other-linktext" href="/chintai/jnc_000065126802/?bc=100232693941" onclick="sendBeaconSiteCatalystClick(event,this,'click_casset_bkn_link',false);" target="_blank">詳細を見る</a></td>

In [69]:
# 賃貸情報
name = each_property.find_all(class_='cassetteitem_content-title')[0].text
address = each_property.find_all(class_='cassetteitem_detail-col1')[0].text
near_station = each_property.find_all(class_='cassetteitem_detail-text')[0].text # 自然言語処理で最寄り駅などの割り出し
built_info = each_property.find_all(class_='cassetteitem_detail-col3')[0].text
built_year = built_info.split('\n')[1]
built_floors = built_info.split('\n')[2]

In [47]:
info_lists = []
for each_property in properties:
    # 賃貸情報
    name = each_property.find_all(class_='cassetteitem_content-title')[0].text
    address = each_property.find_all(class_='cassetteitem_detail-col1')[0].text
    near_station = each_property.find_all(class_='cassetteitem_detail-text')[0].text # 自然言語処理で最寄り駅などの割り出し
    built_info = each_property.find_all(class_='cassetteitem_detail-col3')[0].text
    built_year = built_info.split('\n')[1]
    built_floors = built_info.split('\n')[2]

    # 複数案内がある場合はそれぞれに対して項目を抽出する
    proposes = each_property.find_all(class_='cassetteitem_other')[0]
    # ある一つの案内について
    for each_propose in proposes.find_all('tbody'):
        # 物件情報
        infos = each_propose.find_all('td')
        floor = infos[2].text.split('\r\n\t\t\t\t\t\t\t\t\t\t\t')[1]
        rent_fee = infos[3].find_all(class_='cassetteitem_price--rent')[0].text
        manegement_fee = infos[3].find_all(class_='cassetteitem_price--administration')[0].text
        deposit = infos[4].find_all(class_='cassetteitem_price--deposit')[0].text
        gratuity = infos[4].find_all(class_='cassetteitem_price--gratuity')[0].text
        layout = infos[5].find_all(class_='cassetteitem_madori')[0].text
        exclusive_area = infos[5].find_all(class_='cassetteitem_menseki')[0].text
        detail_link = infos[8].a['href']


        info_list = [name, address, near_station, built_year, built_floors, floor, rent_fee, manegement_fee, deposit, gratuity, layout, exclusive_area, detail_link]
        info_lists.append(info_list)   


In [48]:
len(info_lists)

86

In [50]:
df_rent = pd.DataFrame(info_lists, columns=['name', 'address', 'near_station', 'built_year', 'built_floors', 'floor', 'rent_fee', 'manegement_fee', 'deposit', 'gratuity', 'layout', 'exclusive_area', 'detail_link'])

In [51]:
pd.concat([df, df_rent])

,name,address,near_station,built_year,built_floors,floor,rent_fee,manegement_fee,deposit,gratuity,layout,exclusive_area
0,リモード本厚木Ｂ,神奈川県厚木市温水,小田急小田原線/本厚木駅 バス10分 (バス停)黄金原 歩7分,築34年,2階建,1階,2万円,3000円,-,-,1K,13.7m2
1,リモード本厚木Ｂ,神奈川県厚木市温水,小田急小田原線/本厚木駅 バス10分 (バス停)黄金原 歩7分,築34年,2階建,1階,2万円,5000円,-,-,1K,12.13m2
2,コーポコジマA棟,神奈川県厚木市飯山,小田急小田原線/本厚木駅 バス12分 (バス停)台ノ上 歩3分,築40年,2階建,2階,1.9万円,5000円,-,-,1K,25.2m2
3,コーポコジマA棟,神奈川県厚木市飯山,小田急小田原線/本厚木駅 バス12分 (バス停)台ノ上 歩3分,築40年,2階建,2階,1.9万円,5000円,-,-,1K,25.2m2
4,コーポコジマA棟,神奈川県厚木市飯山,小田急小田原線/本厚木駅 バス12分 (バス停)台ノ上 歩3分,築40年,2階建,2階,1.9万円,5000円,-,-,1K,25.2m2
...,...,...,...,...,...,...,...,...,...,...,...,...
81,LIVE COURT東海大学前,神奈川県秦野市鶴巻南１,小田急小田原線/東海大学前駅 歩6分,築19年,3階建,3階,3.6万円,2000円,-,-,ワンルーム,18.66m2
82,LIVE COURT東海大学前,神奈川県秦野市鶴巻南１,小田急小田原線/東海大学前駅 歩6分,築19年,3階建,2階,4万円,2000円,-,-,1K,20.01m2
83,LIVE COURT東海大学前,神奈川県秦野市鶴巻南１,小田急小田原線/東海大学前駅 歩6分,築19年,3階建,2階,3.4万円,2000円,-,-,ワンルーム,18.66m2
84,LIVE COURT東海大学前,神奈川県秦野市鶴巻南１,小田急小田原線/東海大学前駅 歩6分,築19年,3階建,2階,3.6万円,2000円,-,-,ワンルーム,18.66m2


In [64]:
# 最大ページ
soup.find_all(class_='pagination_set-nav')[0].find_all('li')[-1].text

'2446'

# さらに詳細情報

In [88]:
infos[8].a['href']

'/chintai/jnc_000065126802/?bc=100232693941'

In [81]:
infos[8].a['href']
r_detail = requests.get(f"https://suumo.jp{infos[8].a['href']}")

In [82]:
r_detail

<Response [200]>

In [84]:
soup = BeautifulSoup(r_detail.text, 'lxml')

In [109]:
abstruct_info

[<td class="property_view_table-body" colspan="3">神奈川県相模原市中央区星が丘２</td>,
 <td class="property_view_table-body" colspan="3">
 <div class="property_view_table-read">ＪＲ横浜線/相模原駅 バス12分 (バス停)千代田十字路 歩3分</div>
 <div class="property_view_table-read">ＪＲ相模線/上溝駅 歩17分</div>
 </td>,
 <td class="property_view_table-body">1K</td>,
 <td class="property_view_table-body">17.12m<sup>2</sup></td>,
 <td class="property_view_table-body">築30年</td>,
 <td class="property_view_table-body">4階</td>,
 <td class="property_view_table-body">北東</td>,
 <td class="property_view_table-body">マンション</td>]

In [114]:
# 大まかな情報
abstruct_info = soup.find_all(class_='l-property_view_table')[0].find_all(class_='property_view_table-body')
address = abstruct_info[0].text
near_station = abstruct_info[1].contents # これはfor分で回す
layout = abstruct_info[2].text
exclusive_area = abstruct_info[3].text
built_year = abstruct_info[4].text
built_floors = abstruct_info[5].text
direction = abstruct_info[6].text
built_type = abstruct_info[7].text

In [119]:
# 部屋の特徴
character = soup.find_all(id='bkdt-option')[0].text.split('\n\n\n')[1]

In [141]:
# 物件概要
details = soup.find_all(class_='table_gaiyou')[0].find_all('td')
layout_detail = details[0].text.split('\r\n\t\t\t\t\t\t')[1]
structure = details[1].text.split('\r\n\t\t\t\t\t\t')[1]
built_floors_detail =  details[2].text
built_date = details[3].text
insurance = details[4].text
parking_area = details[5].text
move_in = details[6].text
transaction = details[7].text
condition = details[8].text
store_code = details[9].text
suumo_code = details[10].text
units = details[11].text
previous_update_date = details[12].text
next_update_date = details[13].text
notes = details[14].text

In [154]:
[i.text for i in abstruct_info[1].contents if i!='\n']

['ＪＲ横浜線/相模原駅 バス12分 (バス停)千代田十字路 歩3分', 'ＪＲ相模線/上溝駅 歩17分']

In [157]:
# まとめる

info_lists = []

# 大まかな情報
abstruct_info = soup.find_all(class_='l-property_view_table')[0].find_all(class_='property_view_table-body')

for near_station in [i.text for i in abstruct_info[1].contents if i!='\n']:
    address = abstruct_info[0].text
    layout = abstruct_info[2].text
    exclusive_area = abstruct_info[3].text
    built_year = abstruct_info[4].text
    built_floors = abstruct_info[5].text
    direction = abstruct_info[6].text
    built_type = abstruct_info[7].text
    
    # 部屋の特徴
    character = soup.find_all(id='bkdt-option')[0].text.split('\n\n\n')[1]

    # 物件概要
    details = soup.find_all(class_='table_gaiyou')[0].find_all('td')
    layout_detail = details[0].text.split('\r\n\t\t\t\t\t\t')[1]
    structure = details[1].text.split('\r\n\t\t\t\t\t\t')[1]
    built_floors_detail =  details[2].text
    built_date = details[3].text
    insurance = details[4].text
    parking_area = details[5].text
    move_in = details[6].text
    transaction = details[7].text
    condition = details[8].text
    store_code = details[9].text
    suumo_code = details[10].text
    units = details[11].text
    previous_update_date = details[12].text
    next_update_date = details[13].text
    notes = details[14].text

    info_list = [address, layout, exclusive_area, built_year, built_floors, direction, built_type, character, layout_detail, structure, built_floors_detail, built_date, insurance, parking_area, move_in, transaction, condition, store_code, suumo_code, units, previous_update_date, next_update_date, notes]

    info_lists.append(info_list)

In [159]:
info_lists

[['神奈川県相模原市中央区星が丘２',
  '1K',
  '17.12m2',
  '築30年',
  '4階',
  '北東',
  'マンション',
  '角住戸、プロパンガス、IT重説 対応物件',
  '洋6',
  '鉄筋コン',
  '4階/4階建',
  '1991年11月',
  '要',
  '\r\n\t\t\t\t\t\t-\r\n',
  '即',
  '仲介',
  '\r\n\t\t\t\t\t\t-\r\n',
  'HC1-380124-401-0076',
  '100232693941',
  '20戸',
  '2021/05/30',
  '次回更新日は情報更新日より１５日以内',
  '\n\r\n\t\t\t\t\t\t人気のフローリング、買い物便利！\n\n'],
 ['神奈川県相模原市中央区星が丘２',
  '1K',
  '17.12m2',
  '築30年',
  '4階',
  '北東',
  'マンション',
  '角住戸、プロパンガス、IT重説 対応物件',
  '洋6',
  '鉄筋コン',
  '4階/4階建',
  '1991年11月',
  '要',
  '\r\n\t\t\t\t\t\t-\r\n',
  '即',
  '仲介',
  '\r\n\t\t\t\t\t\t-\r\n',
  'HC1-380124-401-0076',
  '100232693941',
  '20戸',
  '2021/05/30',
  '次回更新日は情報更新日より１５日以内',
  '\n\r\n\t\t\t\t\t\t人気のフローリング、買い物便利！\n\n']]